<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_groundCover_intercepts_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [1]:
# Package and library installation
packages_needed = c("tidyverse", "knitr") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.5     ✔ dplyr   1.0.3
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Remember that the file containing authorization keys for Big Query must be loaded into the virtual envrionment manually.

In [ ]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’




# Source

## Database Connection

In [ ]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [ ]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [ ]:
billing <- bq_test_project()

### vegetation_point_intercept_gridVeg

In [ ]:
con_point_intercept <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_point_intercept_gridVeg",
  billing = billing
)

In [ ]:
dbListTables(con_point_intercept)

[1] "gridVeg_additional_species"         "gridVeg_ground_cover_metadata"     
[3] "gridVeg_image_metadata"             "gridVeg_point_intercept_ground"    
[5] "gridVeg_point_intercept_vegetation" "gridVeg_shrub_tree"                
[7] "gridVeg_survey_metadata"

In [ ]:
ground_point_intercept_sql <- 
"
  SELECT
    survey_ID,
    grid_point,
    intercept_ground_code,
    COUNT(intercept_ground_code) / 2 AS intercepts_pct
  FROM 
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_point_intercept_ground`
  GROUP BY
    survey_ID, grid_point, intercept_ground_code
"
bq_ground_point_intercept <- bq_project_query(billing, ground_point_intercept_sql)
tb_ground_point_intercept <- bq_table_download(bq_ground_point_intercept)
# Drop NA rows. These only occur in intercept_ground_code, are always associated with 
# zero values in intercepts_pct, and will be expanded to true values later using complete()
# df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% drop_na() %>% glimpse() ## LINE COMMENTED OUT; LINE BELOW ADDED; BL; 2021-01-22
df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% filter(intercept_ground_code != "NA") %>% glimpse()

Rows: 9,518
Columns: 4
$ survey_ID             <chr> "69", "69", "69", "234", "234", "234", "234", "…
$ grid_point            <int> 329, 329, 329, 48, 48, 48, 48, 295, 295, 295, 2…
$ intercept_ground_code <chr> "L", "WDT", "BV", "L", "BV", "M/L", "G", "L", "…
$ intercepts_pct        <dbl> 78.0, 1.0, 18.5, 41.5, 19.5, 12.0, 16.5, 51.5, …


### gridVeg_survey_metadata

In [ ]:
survey_metadata_sql <-
"
  SELECT
    survey_ID,
    year,
    survey_sequence
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`
"
bq_survey_metadata <- bq_project_query(billing, survey_metadata_sql)
tb_survey_metadata <- bq_table_download(bq_survey_metadata)
df_survey_metadata <- as.data.frame(tb_survey_metadata) %>% glimpse()

Rows: 1,472
Columns: 3
$ survey_ID       <chr> "F31C56A8-912D-410C-A17D-4C2DD75F71A4", "A19E87E6-A89…
$ year            <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,…
$ survey_sequence <chr> "2016", "2016", "2016", "2016", "2016", "2016", "2016…


year,survey_sequence
<int>,<chr>
2016,2016
2010,2010
2011,2011-12
2012,2011-12
2017,2017
2013,2013
2015,2015


### ground_cover_metadata

In [ ]:
sql_ground_meta <- 
"
SELECT
  intercept_ground_code,
  ground_group
FROM
  `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_ground_cover_metadata`
"
bq_ground_meta <- bq_project_query(billing, sql_ground_meta)
tb_ground_meta <- bq_table_download(bq_ground_meta)
df_ground_meta <- as.data.frame(tb_ground_meta) %>% arrange(intercept_ground_code)

df_ground_meta %>% kable(format = "pandoc")



intercept_ground_code   ground_group 
----------------------  -------------
BG                      inorganic    
BV                      vas_plant    
G                       inorganic    
L                       litter       
LIC                     lichen       
M                       nonvas_plant 
M/L                     nonvas_plant 
OTHER                   inorganic    
R                       inorganic    
S                       inorganic    
SC                      dung         
SD                      dung         
SE                      dung         
SH                      dung         
SU                      dung         
WDL                     woody        
WDS                     woody        
WDSTUMP                 woody        
WDT                     woody        

# Wrangle

In [ ]:
# show unique intercept ground codes
df_ground_point_intercept %>%
  distinct(intercept_ground_code) %>% 
  arrange(intercept_ground_code)

intercept_ground_code
<chr>
BG
BV
G
L
LIC
M
M/L
OTHER
R


## Complete intercept_ground_code
- Complete the data frame for `intercept_ground_code` and fill null cells with 0
- As of 2020-08-24: with 18 ground cover codes and 1244 survey IDs, we should end up with 22,392 rows after completeing the data frame
- We should also end up with 1244 distinct combinations of survey_ID and grid_point

In [ ]:
df_ground_point_intercept_fill <- 
  df_ground_point_intercept %>%
  complete(intercept_ground_code, nesting(survey_ID, grid_point), fill= list(intercepts_pct = 0)) %>%
  glimpse()

df_ground_point_intercept_fill %>% 
  distinct(survey_ID, grid_point) %>% 
  count()

Rows: 23,636
Columns: 4
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG",…
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "013380…
$ grid_point            <int> 285, 505, 401, 23, 323, 511, 402, 196, 226, 564…
$ intercepts_pct        <dbl> 7.0, 4.5, 4.5, 9.0, 0.0, 0.0, 14.0, 18.5, 20.5,…


n
<int>
1244


## Join datasets

In [ ]:
df_gridVeg_groundCover_intercepts <- 
  df_ground_point_intercept_fill %>%
  left_join(df_survey_metadata, by = "survey_ID") %>% 
  left_join(df_ground_meta, by = "intercept_ground_code") %>% 
  select(survey_ID, year, survey_sequence, grid_point, intercept_ground_code, ground_group, intercepts_pct) %>% 
  glimpse()

Rows: 23,636
Columns: 7
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "013380…
$ year                  <int> 2016, 2016, 2017, 2016, 2015, 2016, 2015, 2016,…
$ survey_sequence       <chr> "2016", "2016", "2017", "2016", "2015", "2016",…
$ grid_point            <int> 285, 505, 401, 23, 323, 511, 402, 196, 226, 564…
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG",…
$ ground_group          <chr> "inorganic", "inorganic", "inorganic", "inorgan…
$ intercepts_pct        <dbl> 7.0, 4.5, 4.5, 9.0, 0.0, 0.0, 14.0, 18.5, 20.5,…


## QC
* No rows contain missing data, and this makes sense since `complete()` was used to fill response variables.
* Numeric variables appear correct in the summary

In [ ]:
df_gridVeg_groundCover_intercepts[which(!complete.cases(df_gridVeg_groundCover_intercepts)), ]

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>


In [ ]:
summary(df_gridVeg_groundCover_intercepts)

  survey_ID              year      survey_sequence      grid_point   
 Length:24880       Min.   :2011   Length:24880       Min.   :  1.0  
 Class :character   1st Qu.:2011   Class :character   1st Qu.:136.8  
 Mode  :character   Median :2015   Mode  :character   Median :290.0  
                    Mean   :2014                      Mean   :286.3  
                    3rd Qu.:2016                      3rd Qu.:431.0  
                    Max.   :2017                      Max.   :583.0  
 intercept_ground_code ground_group       intercepts_pct  
 Length:24880          Length:24880       Min.   : 0.000  
 Class :character      Class :character   1st Qu.: 0.000  
 Mode  :character      Mode  :character   Median : 0.000  
                                          Mean   : 4.997  
                                          3rd Qu.: 2.000  
                                          Max.   :99.500  

In [ ]:
df_gridVeg_groundCover_intercepts %>% filter(survey_ID == 234)

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>
234,2011,2011-12,48,BG,inorganic,4.5
234,2011,2011-12,48,BV,vas_plant,19.5
234,2011,2011-12,48,G,inorganic,16.5
234,2011,2011-12,48,L,litter,41.5
234,2011,2011-12,48,LIC,lichen,0.0
234,2011,2011-12,48,M,nonvas_plant,0.0
234,2011,2011-12,48,M/L,nonvas_plant,12.0
234,2011,2011-12,48,OTHER,inorganic,0.0
234,2011,2011-12,48,R,inorganic,3.0


# Output

In [ ]:
# Revision not yet output
# write_csv(df_gridVeg_groundCover_intercepts, path = "gridVeg_groundCover_intercepts_WRANGLE.csv")